# PHASE 1: Sentiment Analysis - Posts in a Chatroom Mentioning a Resource

This script is being used to test supervised learning methods to practice sentiment analysis over chatroom posts, specifically the ones mentioning targeted resources as url links.

The script is being used to have some idea of the feasibility of a ML approach for sentiment analysis for this particular case, as well as to compare some methodologies that could be used in  full implementation later on. 

### METHOD

#### Data Preparation

Data were all posts between May-16 and Mar-17 in the freeCodeCamp Gitter chatroom "HelpBackEnd". The posts selected were those mentioning specified urls or platforms - url selection rules have been mentioned in another section.

As training set I took a fraction of two kaggle competition datasets:

* The dataset from this competition: https://www.kaggle.com/crowdflower/twitter-airline-sentiment, focused on airlines
* The one for this competition: https://inclass.kaggle.com/c/si650winter11, BUT extractedfrom this source http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/, which seemed to be more complete and updated.

No many other comparisons between different possible training datasets have been made so far.

Both datasets were chosen for having enough similarity to the problem faced on this project: we are not trying to compare the sentiment of a conversation: *only the apparent sentiment of the post where the link was mentioned*.

All text at both, training and test datasets, were treated before comparison.

#### Analysis

The sentiment analysis practiced is here was more of **polarity**. The only supervised methods tested so far was **k-Nearest Neighbour (kNN)**. Their selection was motivated by their simplicity. It has been also used for sentiment analysis.

This analysis lacks of related corpus to the data being analysed. It is assumed that the kNN will cope better with that situation, although this assumption is not based on any evaluation.

Posts were compared grouping them by resource they referred to. The sentiment analysis was done only **within** those groups, not between those groups. What it is being used as comparison between groups is the **average sentiment** obtained from the comparison within.

As a general rule, only the similarity to the first 100 neighbours that existed in the training dataset were considered to make the average.

**Be aware that** the analysis was done **without accuracy analysis**. There is no existing annotated test dataset to allow that. Therefore results were taken as *it is*. For this project though **there is an interest in the general trend than in the accuracy/precision of the classification**.


### RESULTS so far...

The results can be commented qualitatively.

### OBSERVATIONS

Some other interested datasets and links to look for:
suggested by @mesmoiron
* https://corplinguistics.wordpress.com/tag/twitter/
* https://gist.github.com/emaadmanzoor/5019020
* http://trec.nist.gov/data/tweets/
* http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/
* http://users.sussex.ac.uk/~johnca/papers/wassa10.pdf
* http://jmcauley.ucsd.edu/data/amazon/

others:
* http://www.sananalytics.com/news/index.php
* https://www.kaggle.com/sonaam1234/sentimentdata
* https://www.crowdflower.com/data-for-everyone/


## References
http://text-processing.com/demo/sentiment/; https://bitbucket.org/japerk/nltk-trainer/src
http://www.nltk.org/howto/sentiment.html
https://www.kaggle.com/ngyptr/python-nltk-sentiment-analysis
http://www.nltk.org/api/nltk.sentiment.html
http://nlpforhackers.io/sentiment-analysis-intro/
https://www.kaggle.com/c/word2vec-nlp-tutorial/data
http://www.markhneedham.com/blog/2013/11/09/python-making-scikit-learn-and-pandas-play-nice/

IN JAVASCRIPT
https://egghead.io/lessons/javascript-compare-similarity-of-strings-through-string-distance-in-natural


https://stackoverflow.com/questions/39682124/using-pandas-and-sklearn-neighbors
https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor/
http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
https://discuss.analyticsvidhya.com/t/how-to-read-zip-file-directly-in-python/1659
http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/
https://stackoverflow.com/questions/38265888/handing-conversion-from-bytes-to-string-when-not-explicitly-opening-a-file-in-py


## Main Python Libraries

In [1]:
import os, sys
import csv, json
#http://blog.revolutionanalytics.com/2016/01/pipelining-r-python.html\n",
import urllib, urllib.request, urllib.parse
import json, re
#from datetime import datetime, date, timedelta
import datetime
import numpy, scipy, pandas
import collections, itertools, copy, operator
import datetime
import zipfile, pickle
import nltk, re
from bs4 import BeautifulSoup
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#sys.path.append('/ec/home/anaconda3/nltk_data')\n",
nltk.data.path.append('/home/ec/anaconda3/nltk_data') #an unfortunate hack for now... need to create a relative link

## Extracting Training and Test Datasets

In [ ]:
directory = config.directory

In [49]:
training = pandas.read_csv(directory+'Tweets.csv')

In [50]:
training.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [51]:
len(data)

14640

In [16]:
raw = pickle.load(open(directory+"helpbackend1_test.pkl", "rb"))


link: www.tutorialspoint.com -- counts: 6, weighted counts: 5; rating: 7.58
link: www.sitepoint.com -- counts: 7, weighted counts: 11; rating: 14.29
link: www.clementinejs.com -- counts: 8, weighted counts: 18; rating: 20.45
link: screencast.com -- counts: 16, weighted counts: 85; rating: 48.30
link: en.wikipedia.org -- counts: 12, weighted counts: 36; rating: 27.27
link: jsfiddle.net -- counts: 7, weighted counts: 26; rating: 33.77
link: docs.npmjs.com -- counts: 13, weighted counts: 41; rating: 28.67
link: developers.google.com -- counts: 6, weighted counts: 17; rating: 25.76
link: www.quandl.com -- counts: 6, weighted counts: 17; rating: 25.76
link: babeljs.io -- counts: 6, weighted counts: 25; rating: 37.88
link: university.mongodb.com -- counts: 7, weighted counts: 34; rating: 44.16
link: mongodb.github.io -- counts: 13, weighted counts: 45; rating: 31.47
link: medium.com -- counts: 30, weighted counts: 128; rating: 38.79
link: www.npmjs.com -- counts: 73, weighted counts: 311; ra

In [4]:
print(len(raw),type(raw))

63200 <class 'list'>


In [5]:
raw[0]

{'fromUser': {'avatarUrl': 'https://avatars-01.gitter.im/gh/uv/3/camperbot',
  'avatarUrlMedium': 'https://avatars1.githubusercontent.com/u/13561988?v=3&s=128',
  'avatarUrlSmall': 'https://avatars1.githubusercontent.com/u/13561988?v=3&s=60',
  'displayName': 'CamperBot',
  'gv': '3',
  'id': '55b977f00fc9f982beab7883',
  'url': '/camperbot',
  'username': 'camperbot',
  'v': 2},
 'html': '<blockquote>\nlegend28469 sends brownie points to <span data-link-type="mention" data-screen-name="valmassoi" class="mention">@valmassoi</span> :sparkles: :thumbsup: :sparkles: </blockquote>\n',
 'id': '57253190e10a59c061077871',
 'issues': [],
 'mentions': [{'screenName': 'valmassoi',
   'userId': '56c11624e610378809c16c8f',
   'userIds': []}],
 'meta': [],
 'readBy': 19,
 'sent': '2016-04-30T22:28:32.114Z',
 'text': '> legend28469 sends brownie points to @valmassoi :sparkles: :thumbsup: :sparkles: ',
 'unread': False,
 'urls': [],
 'v': 1}

In [ ]:
def segment_sentences(words): #words is a LIST, it returns a LIST
    start = 0
    sents = []
    for i, word in enumerate(words):
        #print(i, word)
        if word == '...':
            word = '.'
        if word in '.?!' and classifier_segsen.classify(punct_features(words, i)) == True: #1) if a punctuation mark AND words classified as TRUE 
            sents.append(words[start:i+1])                                          #2) add to sents all word between the start of a sentence and the end
            start = i+1                                                             #3) move to next sentence
    if start < len(words):                                                          #4) if start is still less than the total paragraph..
        sents.append(words[start:])                                                 #5) add all (???)
    return sents   


def subs_codelinksusers(elem):
    soup = BeautifulSoup(elem['html'])
    text = elem['text']
    #code
    codes = soup.find_all('code')
    for c in codes:
        text.replace('```'+c+'```', 'thisiscode')
    
    links = soup.find_all('a')
    for lk in links:
        text.replace('('+lk+')', 'thisislink')
        text.replace('['+lk+']', 'thisislink')
        text.replace(lk, 'thisislink')
    
    for m in elem['mentions']:
        text.replace(m, 'thisisusername')
        

    return text


def links_analysis(raw):
    d = {} #net location : [params, query, user, sent]    
    for elem in raw:
        if elem["fromUser"]["username"] == "camperbot": continue

        for u in elem["urls"]:
            url = u['url']
            try:
                netloc,params,urlq = urllib.parse.urlsplit(url)[1:4]
            except:
                print(url)
                continue

            if netloc in ("files.gitter.im", "gitter.im", "codepen.io", "reddit.com", "www.youtube.com", "github.com"): continue
            if netloc not in list(d.keys()):
                d[netloc] = []
            #d[netloc].append({'params':params, 'urlq':urlq, 'user':elem['fromUser']['username'], 'urls': elem['urls'], 'text': elem['text'], 'sent': elem['sent']})
            d[netloc].append({'params':params, 'urlq':urlq, 'user':elem['fromUser']['username'], 'urls': elem['urls'], 'url1': url, 'text': elem['text'], 'sent': elem['sent']})
    
 
    b = set()
    dd = {} #first_param:{last,count}
    for ll in d.keys():
        for l in d[ll]:
            params = l['params']
            if len(params.split('/')) > 2:
                first_param = ll
                date = l['sent'].split('-')
                year = int(date[0])
                month = int(date[1])
                day = int(date[2].split('T')[0])
                current = datetime.datetime(year, month, day)
                if params.find('FreeCodeCamp')==-1:
                    #print(params)
                    #if params != '':
                    #    b.add(params.split('/')[1])
                    if first_param not in dd:
                        dd[first_param] = {}
                        dd[first_param]['count1'] = 1
                        if year == 2016:
                            dd[first_param]['count2'] = month - 7
                        elif year == 2017:
                            dd[first_param]['count2'] = month + 6
                        dd[first_param]['last'] = current
                        dd[first_param]['repos'] = {}
                    if params not in dd[first_param]['repos']: 
                        dd[first_param]['repos'][params] = {}
                        dd[first_param]['repos'][params]['count1'] = 1
                        dd[first_param]['repos'][params]['texts'] = []
                        dd[first_param]['repos'][params]['users'] = []
                        dd[first_param]['repos'][params]['urls'] = []
                        if year == 2016:
                            dd[first_param]['repos'][params]['count2'] = month - 7
                        elif year == 2017:
                            dd[first_param]['repos'][params]['count2'] = month + 6
                        dd[first_param]['repos'][params]['last'] = current
                        dd[first_param]['repos'][params]['texts'].append(l['text'])
                        dd[first_param]['repos'][params]['users'].append(l['user'])
                        dd[first_param]['repos'][params]['urls'].append(l['urls'])
    
                    if current - dd[first_param]['last'] >= datetime.timedelta(days=1):
                        dd[first_param]['count1'] += 1
                        if year == 2016:
                            dd[first_param]['count2'] += month - 7
                        if year == 2017:
                            dd[first_param]['count2'] += month + 6
                        dd[first_param]['last'] = current
                    if current - dd[first_param]['repos'][params]['last'] >= datetime.timedelta(days=1):
                        dd[first_param]['repos'][params]['count1'] += 1
                        if year == 2016:
                            dd[first_param]['repos'][params]['count2'] += month - 7
                        elif year == 2017:
                            dd[first_param]['repos'][params]['count2'] += month + 6
                        dd[first_param]['repos'][params]['last'] = current                        
                        dd[first_param]['repos'][params]['texts'].append(l['text'])
                        dd[first_param]['repos'][params]['users'].append(l['user'])
                        dd[first_param]['repos'][params]['urls'].append(l['urls'])
        
    for k in sorted(dd, key=lambda k: dd[k]['last'].timestamp(), reverse=False):
        if dd[k]['count1'] > 5:
            print("link: {0} -- counts: {1}, weighted counts: {2}; rating: {3:.2f}".format(k, dd[k]['count1'], dd[k]['count2'], 100*dd[k]['count2']/(11*dd[k]['count1'])))
    
    return d, dd



In [ ]:
test, _ = links_analysis(raw)

In [26]:
scotch_comments = []
for elem in test['scotch.io']:
    scotch_comments.append(elem['text'])

In [27]:
scotch_comments[:10]

['@naprirfan https://scotch.io/tutorials/easy-node-authentication-setup-and-local this is how I did my log in',
 'https://scotch.io/tutorials/easy-node-authentication-setup-and-local',
 '@wordyallen https://scotch.io/tutorials/authenticate-a-node-js-api-with-json-web-tokens',
 'https://scotch.io/tutorials/build-a-restful-api-using-node-and-express-4  <-- using mongoose',
 '@reginaldt these may be helpful: :rabbit: \n* [Mongoose and Node.js tutorial]( http://blog.modulus.io/mongodb-tutorial)\n* [Object Modeling in Node.js with Mongoose](https://devcenter.heroku.com/articles/nodejs-mongoose)\n* [node.js and mongo (using mongoose) tutorial]( http://psitsmike.com/2012/02/node-js-and-mongo-using-mongoose-tutorial/)\n* [Mongoose and Node.js tutorial]( http://theholmesoffice.com/mongoose-and-node-js-tutorial/)\n* [Models]( http://mongoosejs.com/docs/models.html)\n* [mongoose](http://mongoosejs.com/docs/api.html) – do a page search for “model.find”\n* [Queries, References to other documents, S

In [111]:
w3schools_comments = []
for elem in test['www.w3schools.com']:
    w3schools_comments.append(elem['text'])

In [112]:
w3schools_comments[:10]

['@dickorydock these may be useful: :sparkles: \n* [JavaScript getTimezoneOffset() Method](http://www.w3schools.com/jsref/jsref_gettimezoneoffset.asp)\n* [JavaScript: Creating timestamps with time zone offsets](http://weblogs.asp.net/gunnarpeipman/javascript-creating-timestamps-with-time-zone-offsets)\n* [JavaScript: Creating timestamps with time zone offsets](https://dzone.com/articles/javascript-creating-timestamps)\n* [Convert the local time to another time zone with this JavaScript](http://www.techrepublic.com/article/convert-the-local-time-to-another-time-zone-with-this-javascript/)\n* [Localizing Times in JavaScript](https://css-tricks.com/localizing-times-in-javascript/)\n* [How To Convert JavaScript Local Date to UTC And UTC To Local Date](http://praveenlobo.com/blog/how-to-convert-javascript-local-date-to-utc-and-utc-to-local-date/)\n* [Automatic timezone conversion in JavaScript](https://blog.serverdensity.com/automatic-timezone-conversion-in-javascript/)',
 'http://www.tutor

### SELECTING A FRACTION OF TRAINING DATASET (too big) + INVESTIGATING THE DATASET

In [108]:
import random

#original code used IMDB, I was using "Twitter airline sentiment", both on Kaggle
sentiment_data = training.loc[:,["text", "airline_sentiment"]]
sentiment_data = sentiment_data.sample(frac=1)
 
# 80% for training
train_X, train_y = numpy.array(sentiment_data.loc[:9000,['text']]), numpy.array(sentiment_data.loc[:9000,['airline_sentiment']])
 
# Keep 20% for testing
test_X, test_y = numpy.array(sentiment_data.loc[9000:,['text']]), numpy.array(sentiment_data.loc[9000:,['airline_sentiment']])

 


@JetBlue hotspot free WiFi makes me happy. #jfk #itsthelittlethings
@
JetBlue
hotspot
free
WiFi
happy
jfk
itsthelittlethings
1 ['positive']


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag


lemmatizer = WordNetLemmatizer()
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
def clean_text(text):
    print(text)
    text = text.replace("<br />", " ")
    #text = text.decode("utf-8")
 
    return text
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
    text = clean_text(text)
 
 
    raw_sentences = sent_tokenize(text)
    
    
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        
               
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            
           
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            print(lemma)
        
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0
 
# Since we're shuffling, you'll get diffrent results
print(swn_polarity(train_X[0][0]), train_y[0]) # 1 1
#print(swn_polarity(train_X[1]), train_y[1]) # 0 0
#print(swn_polarity(train_X[2]), train_y[2]) # 0 1
#print(swn_polarity(train_X[3]), train_y[3]) # 1 1
#print(swn_polarity(train_X[4]), train_y[4]) # 1 1`
 

In [104]:
numpy.array(list(zip(sentiment_data[:9000])))
train_X[0][0]

'@JetBlue  will do.'

In [101]:
numpy.array(train_y)

array([['neutral'],
       ['negative'],
       ['negative'],
       ..., 
       ['negative'],
       ['negative'],
       ['negative']], dtype=object)

In [82]:
sentiment_data.loc[9000:,['text']].values

array([[ "@USAirways Sadly I'm currently booked on one of your flights tomorrow that I will be changing if I can."],
       [ "See what you started now @nytimes RT @JetBlue: Our fleet's on fleek. http://t.co/atd2Sm8HF4"],
       [ '@united any way you can help me find a flight to LAS? Stuck in DIA, sat on plane 2 hrs, now been waiting an hour for an update.'],
       ..., 
       [ '@SouthwestAir Thanks for the quick reply!  I travel a lot...but not that much, lol.'],
       ['@united what do I do if something is missing from my checked baggage?'],
       [ '@SouthwestAir  Flight today from PHL to DAL Cancelled Flightled. 96 min on hold, rebooked for tomorrow. Now that flight is Cancelled Flightled.  On hold again.']], dtype=object)

In [150]:
import random

X = w3schools_comments

random.shuffle(X)

print(swn_polarity(X[0]))


@UsamaHameed you need sources https://www.w3schools.com/html/html5_audio.asp
@
source
//www.w3schools.com/html/html5_audio.asp
1


## USING OTHER ROOMS

In [21]:
def subs_codelinksusers(elem):
    soup = BeautifulSoup(elem['html'])
    text = elem['text']
    #code
    codes = soup.find_all('code')
    for c in codes:
        text = text.replace('```'+c.text+'```', 'thisiscode')
    
    for u in elem['urls']:
        url = u['url']
        #print(text)
        text = text.replace('('+url+')', 'thisislink')
        text = text.replace('['+url+']', 'thisislink')
        text = text.replace(url, 'thisislink')
    
    for m in elem['mentions']:
        text = text.replace('@'+m['screenName'], 'thisisusername')
        

    return text

room = 'javascript'

directory = "/home/ec/Documents/MainComp_Programming/FreeCodeCamp/data_analysis/1_archive"
raw = pickle.load(open(directory+"/help"+room+"1_test.pkl", "rb"))


## TESTING FOR A FEW EXAMPLES

In [255]:
nl = 'www.sitepoint.com'

rtext = []

for elem in raw:
    for u in elem['urls']:
        url = u['url']
        try:
            netloc,params,urlq = urllib.parse.urlsplit(url)[1:4]
        except:
            print('did not worked', url)
            continue
            
        if netloc == nl:
            rtext.append(subs_codelinksusers(elem))
            break

/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


did not worked http://[\-|\(|\


In [256]:
nl

'www.sitepoint.com'

In [257]:
len(rtext)

62

### BUILDING A SUB-DATASET FOR THE POSTS MENTIONING THE TARGETED URL

In [258]:
subdata = pandas.DataFrame(list(zip(rtext,['neutral']*len(rtext))), columns=['text','airline_sentiment'])
subdata

,text,airline_sentiment
0,Is someone interested in full year premium at ...,neutral
1,thisisusername give this a whirl thisislink,neutral
2,thisisusername \nthisislink\n```js\nvar greetC...,neutral
3,thisisusername i had a similar problem when wo...,neutral
4,thisisusername here's an article if you want t...,neutral
5,"thisisusername if you're stll interested, this...",neutral
6,thisisusername [this]thisislink has some good ...,neutral
7,you need to be using that in an ajax request ...,neutral
8,thisisusername you may find the following usef...,neutral
9,"thisisusername you may find this useful: ""Aja...",neutral


In [259]:
#data = pandas.read_csv('/home/ec/Documents/MainComp_Programming/FreeCodeCamp/data_analysis/1_archive/datascienceroom/Tweets.csv')
training = training.loc[:,['text','airline_sentiment']]

datalen = len(training)

h = pandas.concat([training,subdata], ignore_index=True)
h['index1'] = h.index
datalen

14640

In [260]:
import zipfile
import csv
#data = pandas.read_csv('/home/ec/Documents/MainComp_Programming/FreeCodeCamp/data_analysis/1_archive/datascienceroom/Tweets.csv')
zf = zipfile.ZipFile('/home/ec/Documents/MainComp_Programming/FreeCodeCamp/data_analysis/1_archive/datascienceroom/Sentiment-Analysis-Dataset.zip') # having First.csv zipped file.
#data = pandas.read_csv(zf.open('Sentiment Analysis Dataset.csv'))
counter = 0
d = []
with zf.open('Sentiment Analysis Dataset.csv', 'r') as f:
    csvfile = (line.decode('utf8') for line in f)
    freader = csv.reader(csvfile)
    #while 1:
    next(freader, None)
    for row in freader:
        #print(row)
        #print(freader.readline())
        if len(row) > 4:
            print(counter)
            continue
        d.append(row)
        counter += 1
        if counter > 100000:
            break

8834


In [261]:
data = pandas.DataFrame(d, columns=['x','airline_sentiment','ss','text'])
data

data = data.loc[:,['text','airline_sentiment']]

datalen = len(data)

h = pandas.concat([data,subdata], ignore_index=True)
h['index1'] = h.index
datalen

100001

In [262]:
data.head()

,text,airline_sentiment
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0


In [263]:
# loading library
from sklearn.feature_extraction.text import CountVectorizer

#n_samples = 2000
n_features = 5000
#n_topics = 10
#n_top_words = 20


def knn(textsarray):
     pass
    


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')

bow_vec = vectorizer.fit_transform(h['text'])

vectorizer.get_feature_names()

from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors( n_neighbors=2, algorithm='ball_tree' ) #no matter what I use, it is always the same
#modelfit = model.fit(count_vect, enc)
modelfit = model.fit(bow_vec)

#le = LabelEncoder()
#neigh = KNeighborsClassifier(n_neighbors=5)
#neigh.fit(bow_vec, data['airline_sentiment'])
#neigh.predict()


/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/neighbors/base.py:211: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [264]:
#sentestimate = []
try:
    for i in modelfit.kneighbors( bow_vec[datalen+2], n_neighbors=100, return_distance=False ):
        #print(h.ix[i, ['text','airline_sentiment','index1']])
        #print(i)
        print(h.ix[i, ['text']].values)
        #sentestimate = h.ix[i, ['airline_sentiment']].values
        #print(modelfit.kneighbors( bow_vec[datalen+2], n_neighbors=100, return_distance=False ).get_value(i,'index1'))
        #h.get_value(i, 'airline_sentiment')
except:
    print('HELLO, an error ocurred!!')

[[ 'thisisusername \nthisislink\n```js\nvar greetCurried = function(greeting) {\n  return function(name) {\n    console.log(greeting + ", " + name);\n  };\n};\n\nvar greetHello = greetCurried("Hello");\ngreetHello("Heidi"); //"Hello, Heidi"\ngreetHello("Eddie"); //"Hello, Eddie"\n``` ']
 ['@bayanbay2 hello bayan and @Sharasandiboom hello shara ']
 ['@clovertised hello. are you finee ']
 ['@alcofrolic_x hello ']
 ['@AYellowHouse Hello! ']
 ['@charmdube why hello ']
 ['@coeurenbois - hello to you ']
 ['@CoverFX Hello!  How are you? ']
 ['@amateur26 Hello to you too ']
 ['@cmglothlin Hello. ']
 ['@chaddotcom hello ']
 ['@afreshmusic Hello You!! ']
 ['@brokheart hello ']
 ['@angelaharms hello then ']
 ['@CoachCharrise Charrise!! Hello! ']
 ['@badavarascal hello - has your 500D landed? ']
 ['@Amaninazaini  why hello thar.']
 ['@beccatheirish1 Well hello there shizzlebisc-orino ']
 ['@Comedyscreen  Hello']
 ['@chrisgbsb Hello! How are you?! ']
 ['@arghkat HELLO ']
 ['@choose2befit meow, hell

In [265]:
#sentestimate

In [266]:
#for ii,s in enumerate(sentestimate):
#    if s[0] == 'positive':
#        sentestimate[ii] = 1
#    if s[0] == 'negative':
#        sentestimate[ii] = -1
#    if s[0] == 'neutral':
#        sentestimate[ii] = 0 

In [267]:
#sentestimate

In [268]:
#sentestimate.mean()

In [269]:
bow_vec.shape

(100063, 5000)

In [270]:
## fitting the model
#knn.fit(X_train, y_train)

## predict the response
#pred = knn.predict(X_test)

## evaluate accuracy
#print accuracy_score(y_test, pred)  

In [271]:
allsent = []

#https://stackoverflow.com/questions/19852586/get-mean-value-avoiding-nan-using-numpy-in-python
#https://stackoverflow.com/questions/18200052/how-to-convert-ndarray-to-array
#https://stackoverflow.com/questions/10377096/multiple-conditions-using-or-in-numpy-array

numneig = len(rtext)+100
#numneig = 600

for n in range(len(rtext)):
    for indices in modelfit.kneighbors( bow_vec[datalen+n], n_neighbors= numneig, return_distance=False ):
        #print(h.ix[indices, ['text']].values[0])
        sentestimate = h.ix[indices, ['airline_sentiment']].values
        for ii,s in enumerate(sentestimate):
            if s[0] == 'positive' or s[0] == '1':
                sentestimate[ii] = 1*(len(rtext)+100 - ii)
            if s[0] == 'negative' or s[0] == '0':
                sentestimate[ii] = -1*(len(rtext)+100 - ii)
            if s[0] == 'neutral':
                sentestimate[ii] = 0
                #sentestimate[ii] = numpy.nan
        #print(sentestimate.mean())
        #allsent.append(sentestimate.mean())
        #print(sentestimate.reshape(-1))
        anotherarray = sentestimate.reshape(-1)
        anotherarray = sentestimate.ravel()
        #allsent.append(numpy.nanmean(anotherarray))
        #print(anotherarray[numpy.nanmean(anotherarray)])
        #print(anotherarray[~numpy.isnan(anotherarray)])
        print(anotherarray[numpy.where(anotherarray != 0)][:100].mean())
        val = anotherarray[numpy.where(anotherarray != 0)][:100].mean()
        allsent.append(val)
        #print(anotherarray[numpy.where(anotherarray != 0)].mean()/((len(rtext)+100)))
allsent = numpy.array(allsent)
#allsent = allsent[numpy.where((allsent >= 20)|(allsent <= -20))]
print(allsent)
if len(allsent) == 0:
    allsent = numpy.array([0])
print('url: {0};\nNum. of msgs: {1};\nsentiment rating: {2:.1f}'.format(nl, len(rtext), sum(allsent)/len(allsent)/10))

15.33
17.08
101.08
31.13
-27.25
12.81
25.18
15.08
21.79
17.2
4.8
15.11
20.3
24.66
-24.57
25.44
14.31
-2.22
-46.97
4.93
17.08
10.02
14.65
16.87
-8.2
24.86
21.04
17.08
36.89
13.66
6.88
13.32
36.37
16.46
16.46
17.27
17.08
38.1
17.08
-39.47
17.08
14.26
-11.89
19.72
40.99
16.46
32.17
17.08
-3.59
16.46
5.3
19.29
29.12
8.96
17.09
20.12
6.48
-43.6
16.46
17.08
19.76
20.76
[  15.33   17.08  101.08   31.13  -27.25   12.81   25.18   15.08   21.79
   17.2     4.8    15.11   20.3    24.66  -24.57   25.44   14.31   -2.22
  -46.97    4.93   17.08   10.02   14.65   16.87   -8.2    24.86   21.04
   17.08   36.89   13.66    6.88   13.32   36.37   16.46   16.46   17.27
   17.08   38.1    17.08  -39.47   17.08   14.26  -11.89   19.72   40.99
   16.46   32.17   17.08   -3.59   16.46    5.3    19.29   29.12    8.96
   17.09   20.12    6.48  -43.6    16.46   17.08   19.76   20.76]
url: www.sitepoint.com;
Num. of msgs: 62;
sentiment rating: 1.4


In [272]:
#'Anyone looking to learn JavaScript ES6 consider following this series: https://medium.com/@__Masha__/learn-es6-the-dope-way-i-const-let-var-ae828580472b#.pfurdplqu'.find('medium') 

# CODE BELOW UNDER REVISION - DUMPED

In [31]:
     
data = pandas.read_csv("/home/ec/Documents/MainComp_Programming/FreeCodeCamp/data_analysis/1_archive/datascienceroom/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
 
## 25000 movie reviews
print(data.shape) # (25000, 3) 
print(data["review"][0])         # Check out the review
print(data["sentiment"][0])          # Check out the sentiment (0/1)
 

(25000, 3)
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it 

In [12]:
raw[5]

{'fromUser': {'avatarUrl': 'https://avatars-01.gitter.im/gh/uv/3/DanStockham',
  'avatarUrlMedium': 'https://avatars2.githubusercontent.com/u/11827054?v=3&s=128',
  'avatarUrlSmall': 'https://avatars2.githubusercontent.com/u/11827054?v=3&s=60',
  'displayName': 'Dan Stockham',
  'gv': '3',
  'id': '5586c26215522ed4b3e23f3a',
  'url': '/DanStockham',
  'username': 'DanStockham',
  'v': 12},
 'html': 'Has anyone have any trouble with <code>res.redirect()</code> with express?',
 'id': '572554dd60e2f3e873665b5e',
 'issues': [],
 'mentions': [],
 'meta': [],
 'readBy': 16,
 'sent': '2016-05-01T00:59:09.070Z',
 'text': 'Has anyone have any trouble with ```res.redirect()``` with express?',
 'unread': False,
 'urls': [],
 'v': 1}

In [20]:
soup = BeautifulSoup(raw[5]['html'])

/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [27]:
print(type(soup.find('code').text))

<class 'str'>


In [29]:
raw[5]['text'].replace('```'+soup.find('code').text+'```','thisiscode')

'Has anyone have any trouble with thisiscode with express?'

In [38]:
for elem in raw[5:]:
    if BeautifulSoup(elem['html']).find_all('a'):
        print(elem['html'])
        print(elem['text'])
        print(len(BeautifulSoup(elem['html']).find_all('a')))
        break

<blockquote>
:star: 403 | <span data-link-type="mention" data-screen-name="valmassoi" class="mention">@valmassoi</span> | <a href="http://www.freecodecamp.com/valmassoi" rel="nofollow" target="_blank" class="link">http://www.freecodecamp.com/valmassoi</a></blockquote>

> :star: 403 | @valmassoi | [http://www.freecodecamp.com/valmassoi](http://www.freecodecamp.com/valmassoi)
1


/home/ec/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
